In [72]:
from pylab import *
from lego_robot import *

In [73]:
def compute_derivative(scan, min_dist):
    jumps = [ 0 ]
    for i in range(1, len(scan) - 1):
        # --->>> Insert your code here.
        # Compute derivative using formula "(f(i+1) - f(i-1)) / 2".
        l = scan[i-1]
        r = scan[i+1]
        
        # Do not use erroneous scan values, which are below min_dist.
        if (l > min_dist and r > min_dist): 
            der = (r-l)/2.0
            jumps.append(der) # Replace this line, append derivative instead.
        else: 
            jumps.append(0)
    return jumps

In [74]:
def find_cylinders(scan, scan_derivative, jump, min_dist):
    cylinder_list = []
    on_cylinder = False
    sum_rays, sum_depths, rays = 0.0, 0.0, 0

    for i in range(len(scan_derivative)):
        # --->>> Insert your cylinder code here.
        # Whenever you find a cylinder, add a tuple
        # (average_ray, average_depth) to the cylinder_list.
        if (abs(scan_derivative[i]) > jump): #trigger condition
            if (scan_derivative[i] < 0): #left edge
                if (on_cylinder == False): #on cylinder
                    on_cylinder = True
                else: #LL
                    sum_rays = 0.0
                    sum_depths = 0.0
                    rays = 0
                        
            else: #right edge
                if (on_cylinder == True): #ignore RR
                    on_cylinder = False
                    cylinder_list.append((sum_rays/rays,sum_depths/rays))
                    sum_rays = 0.0
                    sum_depths = 0.0
                    rays = 0
                    
        if (on_cylinder == True):
            if (scan[i]>min_dist):
                rays += 1
                sum_rays += i
                sum_depths += scan[i]

    return cylinder_list

In [75]:
def compute_cartesian_coordinates(cylinders, cylinder_offset):
    result = []
    for (bindex,rangec) in cylinders:
        # --->>> Insert here the conversion from polar to Cartesian coordinates.
        # c is a tuple (beam_index, range).
        # For converting the beam index to an angle, use
        # LegoLogfile.beam_index_to_angle(beam_index)
        index_as_radians = LegoLogfile.beam_index_to_angle(bindex)
        x = cos(index_as_radians) * (rangec + cylinder_offset)
        y = sin(index_as_radians) * (rangec + cylinder_offset)
        result.append( (x,y) ) # Replace this by your (x,y)
    return result

In [76]:
logfile = LegoLogfile()
logfile.read("robot4_scan.txt")

In [77]:
minimum_valid_distance = 25.0
depth_jump = 100.0

In [78]:
scan_no = 7
scan = logfile.scan_data[scan_no]

In [79]:
der = compute_derivative(scan, minimum_valid_distance)

In [80]:
cylinders = find_cylinders(scan, der, depth_jump, minimum_valid_distance)

In [81]:
cylinders

[(232.0, 374.93548387096774),
 (289.5, 1403.2),
 (364.0, 1668.111111111111),
 (416.5, 1176.75),
 (476.5, 713.3),
 (499.5, 1505.5)]

In [82]:
    minimum_valid_distance = 20.0
    depth_jump = 100.0
    cylinder_offset = 90.0

    # Read the logfile which contains all scans.
    logfile = LegoLogfile()
    logfile.read("robot4_scan.txt")

    # Write a result file containing all cylinder records.
    # Format is: D C x[in mm] y[in mm] ...
    # With zero or more points.
    # Note "D C" is also written for otherwise empty lines (no
    # cylinders in scan)
    out_file = open("cylinders.txt", "w")
    for scan in logfile.scan_data:
        # Find cylinders.
        der = compute_derivative(scan, minimum_valid_distance)
        cylinders = find_cylinders(scan, der, depth_jump,
                                   minimum_valid_distance)
        cartesian_cylinders = compute_cartesian_coordinates(cylinders,
                                                            cylinder_offset)
        # Write to file.
       
        out_file.write("D C ")
        for c in cartesian_cylinders:
            out_file.write("%.1f %.1f " % c)
        out_file.write("\n")
    out_file.close()
